In [1]:
from __future__ import annotations

from prompts import PROBLEM_PROMPT
from orchestrator import Orchestrator
from dev import DevAgent
from summary_agent import SummaryAgent


def run_iterations(n: int = 20) -> str:
    blackboard = ""
    agent = DevAgent()
    summarizer = SummaryAgent()

    for iteration in range(n):
        # 1) Orchestrate (pass total budget so it can plan)
        orch = Orchestrator(
            problem_description=PROBLEM_PROMPT,
            blackboard=blackboard,
            iteration=iteration,
            max_iterations=n,  # NEW: total task budget
        )
        step = orch.orchestrator_step()  # expected: {"text": "...", ...}
        # print("Orchestrator instructions: ", step['text'])
        # print("---") 
        instr = step["text"] if isinstance(step, dict) else step

        # 2) Dev agent executes
        report = agent.run(instr, iteration=iteration, max_repairs=3)

        # 3) Summarize and update blackboard
        out = summarizer.run(
            iteration=iteration,
            orchestrator_text=instr,
            current_blackboard=blackboard,
            dev_report=report,
            save_path=None,
        )

        print(f"\n=== Summary: Iteration {iteration} ===\n")
        print(out["summary_block"])
        blackboard = out["updated_blackboard"]

    return blackboard


if __name__ == "__main__":
    final_blackboard = run_iterations(5)
    print("\n=== Final Blackboard ===\n")
    print(final_blackboard)


=== API RESPONSE DEBUG ===
Response choices length: 1
First choice finish_reason: stop
Message content is None: False
Message content length: 6394
=== DEV AGENT SYSTEM PROMPT ===
You are the Dev Agent.

Allowed files:
- ./train.csv, ./test.csv

Feature columns in test.csv:
id, spacegroup, number_of_total_atoms, percent_atom_al, percent_atom_ga, percent_atom_in,
lattice_vector_1_ang, lattice_vector_2_ang, lattice_vector_3_ang,
lattice_angle_alpha_degree, lattice_angle_beta_degree, lattice_angle_gamma_degree

Objective:
Predict for each material id in test.csv:
- formation_energy_ev_natom
- bandgap_energy_ev

Evaluation:
Column-wise RMSLE with log1p:
RMSLE = sqrt( (1/n) * ...
=== DEV AGENT LLM RESPONSE ===
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

def initial_eda():
    """
    Performs an initial exploratory data analysis on the train and test datasets.
    """
    # Create output directory
    output_dir = './0/'
    if not os.path.exis

/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/sklearn/metrics/_ranking.py:424: UndefinedMetricWarning: On

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000687 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2148
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 13
[LightGBM] [Info] Start training from score 1.070127
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000783 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2153
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 13
[LightGBM] [Info] Start training from score 1.067053
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001016 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2153
[LightGBM] [Info] Number of data points in the 

/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/joblib/externals/loky/backend/resource_tracker.py:299: UserWarning: resource_tracker: There appear to be 18 leaked semlock objects to clean up at shutdown
  warnings.warn(
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/joblib/externals/loky/backend/resource_tracker.py:299: UserWarning: resource_tracker: There appear to be 6 leaked folder objects to clean up at shutdown
  warnings.warn(
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/joblib/externals/loky/backend/resource_tracker.py:315: UserWarning: resource_tracker: /var/folders/8n/lyy8fgpj1c167_855184l99w0000gn/T/joblib_memmapping_folder_32067_17c525ff19f34b5ca9d4839e9dd2330c_d9cfc26bc9f743c8aba4dcb766df3c2f: FileNotFoundError(2, 'No such file or directory')
  warnings.warn(f"resource_tracker: {name}: {e!r}")
/Users/atharvjain/code/ascent/.venv/lib/python3.13/site-packages/joblib/externals/loky/backend/resource_tracker.py:315: UserWarn

In [3]:
# This is just for testing purposes

from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

comp = api.competition_view("playground-series-s5e5")
print(comp.title)
print(comp.evaluationMetric)
print(comp.description[:500])  # first 500 chars of description


AttributeError: 'KaggleApi' object has no attribute 'competition_view'